In [7]:
import os
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict

def dhash(image, hash_size=8):
    """Tính toán logic nội dung ảnh - ảnh giống nhau sẽ có cùng mã hash này"""
    image = image.convert('L').resize((hash_size + 1, hash_size), Image.Resampling.LANCZOS)
    pixels = np.asarray(image)
    diff = pixels[:, 1:] > pixels[:, :-1]
    return "".join(["%1x" % sum([2**i for i, v in enumerate(row) if v]) for row in diff])

def get_image_paths(directory):
    directory = Path(directory)
    # Lọc bỏ file lặp do Windows case-insensitive
    extensions = ("*.jpg", "*.jpeg", "*.png", "*.webp", "*.JPG", "*.PNG")
    paths = set()
    for ext in extensions:
        for p in directory.glob(ext):
            paths.add(p.resolve())
    return list(paths)

def deep_clean_content():
    base_dir = Path(r"d:\Computer Vision\Computer-Vision Project\Computer-Vision-\data\images")
    # Kiểm tra xem folder test có viết hoa hay không
    folders = ["normal", "incident", "test"]
    
    threshold = 1 # Ngưỡng nội dung (1 bit lệch vẫn coi là trùng)
    all_hashes = defaultdict(list)
    
    print("--- Bước 1: Đang quét nội dung ảnh ---")
    for f_name in folders:
        folder_path = base_dir / f_name
        if not folder_path.exists(): 
            print(f"hông tìm thấy folder: {f_name}")
            continue
        
        paths = get_image_paths(folder_path)
        for path in tqdm(paths, desc=f"Scanning content in {f_name}"):
            try:
                with Image.open(path) as img:
                    h = dhash(img)
                    all_hashes[h].append(path)
            except: pass

    to_delete = []

    print("\n--- Bước 2: Phân tích ảnh trùng nội dung ---")
    for h, paths in all_hashes.items():
        if len(paths) > 1:
            # Sắp xếp để giữ lại ảnh tốt nhất:
            # Ưu tiên: Traing (normal/incident) > Test
            paths.sort(key=lambda p: (1 if "test" not in str(p).lower() else 0, -len(str(p))))
            
            original = paths[-1] # Đây là ảnh giữ lại
            duplicates = paths[:-1] # Đây là những ảnh trùng nội dung cần xóa
            
            for dup in duplicates:
                print(f"Phát hiện trùng nội dung: {dup.parent.name}/{dup.name} <== GIỐNG ==> {original.parent.name}/{original.name}")
                to_delete.append(dup)

    if not to_delete:
        print("\nTuyệt vời! Tất cả ảnh đều có nội dung khác nhau.")
        return

    print(f"\nTổng cộng có {len(to_delete)} ảnh bị trùng nội dung.")
    confirm = input(f"Bạn có muốn XÓA các bản trùng này để sạch data không? (y/n): ")
    
    if confirm.lower() == 'y':
        for path in to_delete:
            try:
                os.remove(path)
                # print(f"Đã xóa: {path.name}")
            except: pass
        print(f"\nĐã xóa xong {len(to_delete)} ảnh trùng nội dung!")
    else:
        print("\nĐã hủy lệnh xóa.")

if __name__ == "__main__":
    deep_clean_content()

--- Bước 1: Đang quét nội dung ảnh ---


Scanning content in incident: 100%|██████████| 91/91 [00:00<00:00, 149.00it/s]
Scanning content in test: 0it [00:00, ?it/s]


--- Bước 2: Phân tích ảnh trùng nội dung ---

Tuyệt vời! Tất cả ảnh đều có nội dung khác nhau.
